In [2]:
import requests
import json
import pandas as pd
from time import sleep

err_count = 0
while err_count < 3:
    try:
        #小心requrest太過頻繁IP會被對方封鎖
        res1 = requests.get('http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20170901&stockNo=2330')
        sleep(5)
        res2 = requests.get('http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20171001&stockNo=2330')
        sleep(5)
        res3 = requests.get('http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20171101&stockNo=2330')
        break
    except:
        sleep(5)
        err_count += 1
        continue
if err_count == 3:
    print('連線失敗')

json1 = res1.json()
json2 = res2.json()
json3 = res3.json()

In [3]:
header = json1['fields']
header

['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']

In [4]:
data = json1['data']
data.extend(json2['data'])
data.extend(json3['data'])
len(data)

63

In [5]:
df = pd.DataFrame.from_records(data, columns=header)
df.sort_index(ascending=False).head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
62,106/11/30,"112,262,770","25,539,072,296",226.50,230.50,226.00,226.00,-8.50,"21,352"
61,106/11/29,"26,535,797","6,234,161,137",235.00,236.00,234.50,234.50,+0.50,"6,976"
60,106/11/28,"47,834,178","11,198,457,400",234.50,235.00,233.00,234.00,-3.00,"15,262"
59,106/11/27,"32,247,327","7,696,717,391",240.00,240.00,237.00,237.00,-7.00,"14,743"
58,106/11/24,"11,404,453","2,777,319,771",242.00,244.50,241.50,244.00,+1.50,"3,910"


In [6]:
#日期轉換西元年、如使用Jupyter重複執行這裡會讓日期溢位
for i, row in df.iterrows():
    #print(row[0])
    strDate = row[0]
    DateArr = strDate.split("/")
    DateArr[0] = str(int(DateArr[0])+1911)
    DateArr[1] = str(int(DateArr[1]))
    DateArr[2] = str(int(DateArr[2]))
    #df.set_value(i,'日期','/'.join(DateArr))
    df.loc[i, '日期'] = '/'.join(DateArr)
df.sort_index(ascending=False).head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
62,2017/11/30,"112,262,770","25,539,072,296",226.50,230.50,226.00,226.00,-8.50,"21,352"
61,2017/11/29,"26,535,797","6,234,161,137",235.00,236.00,234.50,234.50,+0.50,"6,976"
60,2017/11/28,"47,834,178","11,198,457,400",234.50,235.00,233.00,234.00,-3.00,"15,262"
59,2017/11/27,"32,247,327","7,696,717,391",240.00,240.00,237.00,237.00,-7.00,"14,743"
58,2017/11/24,"11,404,453","2,777,319,771",242.00,244.50,241.50,244.00,+1.50,"3,910"


In [7]:
df.to_csv('../data/fp_demo.csv', sep=',', encoding='utf-8-sig', index=False)

In [8]:
import sqlite3 as sql

#連線至sqlite檔案，若無該檔案sql，則會建立一個新的
conn = sql.connect("../data/twse.db")
#將Dataframe資料寫入sql檔中的'demo1'表中，無該資料表則會自動建立
df.to_sql("demo1", conn, if_exists="replace", index=False)

In [10]:
pd.read_sql_query("select * from demo1;", conn).sort_index(ascending=False)

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
62,2017/11/30,"112,262,770","25,539,072,296",226.50,230.50,226.00,226.00,-8.50,"21,352"
61,2017/11/29,"26,535,797","6,234,161,137",235.00,236.00,234.50,234.50,+0.50,"6,976"
60,2017/11/28,"47,834,178","11,198,457,400",234.50,235.00,233.00,234.00,-3.00,"15,262"
59,2017/11/27,"32,247,327","7,696,717,391",240.00,240.00,237.00,237.00,-7.00,"14,743"
58,2017/11/24,"11,404,453","2,777,319,771",242.00,244.50,241.50,244.00,+1.50,"3,910"
57,2017/11/23,"9,562,763","2,320,934,646",242.00,243.50,241.50,242.50,+0.50,"4,253"
56,2017/11/22,"27,251,431","6,631,376,152",244.50,244.50,242.00,242.00,-1.00,"7,655"
55,2017/11/21,"22,878,024","5,529,803,248",240.00,243.00,239.50,243.00,+4.50,"7,942"
54,2017/11/20,"19,811,271","4,744,311,377",241.00,241.50,238.00,238.50,-3.00,"5,935"
53,2017/11/17,"18,741,825","4,528,440,917",240.50,242.50,240.50,241.50,+3.50,"5,647"
